# Simple Field Benchmarks
Run simple dipole test case (with few loops) to make sure our reorganization is working properly. Test out our emission model for a modest number of loops to make sure everything is working properly and scaling as expected.

In [1]:
import os
import subprocess
import time
import json

import dask
import dask.distributed
import numpy as np
import scipy.interpolate
import astropy.units as u
from astropy.utils.console import ProgressBar
import matplotlib.pyplot as plt
import matplotlib.colors
import seaborn
from sunpy.map import Map,GenericMap
from sunpy.instr.aia import aiaprep
import fiasco
import synthesizAR
import synthesizAR.maps
from synthesizAR.model_ext import UniformHeating, EbtelInterface
from synthesizAR.instruments import InstrumentSDOAIA
from synthesizAR.atomic import Ion,Element,EmissionModel,list_elements
from solarbextrapolation.example_data_generator import dummyDataToMap

%matplotlib inline

In [2]:
client = dask.distributed.Client()
client

Client Scheduler: tcp://127.0.0.1:35619 Dashboard: http://127.0.0.1:8787,Cluster Workers: 64 Cores: 64 Memory: 270.38 GB


## Synthetic HMI Map
First, make a basic dipole map.

In [ ]:
def make_synthetic_map(xrange,yrange,shape,spots):
    delta_x = xrange[1] - xrange[0]
    delta_y = yrange[1] - yrange[0]
    dx = delta_x/arr_grid_shape[0]
    dy = delta_y/arr_grid_shape[1]
    synthetic_hmi_data = np.zeros(arr_grid_shape)
    xphysical,yphysical = np.meshgrid(np.arange(arr_grid_shape[0])*dx,np.arange(arr_grid_shape[1])*dy)
    for spot in spots:
        if spot[0].unit is u.percent:
            pos = u.Quantity([spot[0][0].value/100.*delta_x + xrange[0],
                              spot[0][1].value/100.*delta_y + yrange[0]])
        else:
            pos = spot[0]
        if spot[1].unit is u.percent:
            sigma = spot[1].value/100.*delta_x
        else:
            sigma = spot[1]
        An_max   = spot[2].value
        An_x     = pos[0]
        An_y     = pos[1]
        An_Dx    = xphysical - An_x + xrange[0]
        An_Dy    = yphysical - An_y + yrange[0]
        An_DxSqu = An_Dx.value**2.0
        An_DySqu = An_Dy.value**2.0
        An_Sigma = sigma.value

        # So this contibution is calculated and added.
        synthetic_hmi_data += An_max * np.exp( - ( (An_DxSqu + An_DySqu) / (2 * An_Sigma**2.0)) )
        
    synthetic_hmi_map = dummyDataToMap(synthetic_hmi_data,xrange,yrange)
    test = Map('/storage-home/w/wtb2/sunpy/data/hmi_m_45s_2011_02_12_15_32_15_tai_magnetogram.fits')
    for key in ['ctype1','ctype2','rsun_ref','rsun_obs','dsun_ref','dsun_obs',
                'wcsname','crlt_obs','t_obs','date-obs']:
        synthetic_hmi_map.meta[key] = test.meta[key]
        
    return synthetic_hmi_map

In [ ]:
arr_grid_shape = [ 300, 300]
xrange = u.Quantity([ -150.0, 150.0 ], u.arcsec)
yrange = u.Quantity([ -150.0, 150.0 ], u.arcsec)
spots = [
    ( u.Quantity([65,50],u.percent), 5*u.percent, -5e3*u.Gauss),
    ( u.Quantity([35,50],u.percent), 5*u.percent,  5e3*u.Gauss)
]

In [ ]:
hmi_map = make_synthetic_map(xrange,yrange,arr_grid_shape,spots)

## Build Field
Assemble the extrapolated field and trace streamlines.

In [ ]:
field = synthesizAR.Skeleton(hmi_fits_file=hmi_map,resample=u.Quantity([100,100],'pix'))

In [ ]:
zrange = u.Quantity([0,300],'arcsec')
zshape = 100

In [ ]:
field.extrapolate_field(zshape,zrange)

In [ ]:
field.extract_streamlines(100,
                          close_threshold=0.05,
                          mask_threshold=0.2,
                          safety=2,
                          loop_length_range=[1e9,2e10]*u.cm,
                          direction=-1)

In [ ]:
field.peek()

In [ ]:
field.make_loops()

In [ ]:
plt.hist([loop.full_length.to(u.Mm).value for loop in field.loops],bins='scott',histtype='step',lw=2);
plt.xscale('log')

## Loop Models

In [ ]:
class SingleEventModel(UniformHeating):
    @property
    def number_events(self):
        return 1
heating_options = {'duration':200,'average_waiting_time':1000,'duration_rise':100,
                   'duration_decay':100,'stress_level':1e-6}
heating_model = SingleEventModel(heating_options)

In [ ]:
base_config = synthesizAR.util.InputHandler(
    '/storage-home/w/wtb2/codes/ebtelPlusPlus/config/ebtel.example.cfg.xml'
).lookup_vars()
base_config['total_time'] = 5e3
base_config['use_adaptive_solver'] = True
base_config['use_flux_limiting'] = True
base_config['force_single_fluid'] = False
base_config['heating']['partition'] = 1.
base_config['heating']['background'] = 1e-6

In [ ]:
ebtel_interface = EbtelInterface(base_config,
                                 heating_model,
                                 '/storage-home/w/wtb2/data/simple_ar_model/hydro_config/',
                                 '/storage-home/w/wtb2/data/simple_ar_model/hydro_results/')

In [ ]:
field.configure_loop_simulations(ebtel_interface)

In [ ]:
def ebtel_runner(loop):
    subprocess.call([os.path.join('/storage-home/w/wtb2/codes/','ebtelPlusPlus/bin/ebtel++.run'),
                     '-c',loop.hydro_configuration['config_filename']])

In [ ]:
for loop in field.loops:
    ebtel_runner(loop)

In [ ]:
field.load_loop_simulations(ebtel_interface,savefile='/storage-home/w/wtb2/data/simple_ar_model/loop_parameters.h5')

In [ ]:
fig,axes = plt.subplots(2,1,figsize=(20,10),sharex=True)
plt.subplots_adjust(hspace=0.)
for loop in field.loops[::]:
    axes[0].plot(loop.time,np.max(loop.electron_temperature.to(u.MK),axis=1),color='b',alpha=0.1)
    axes[0].plot(loop.time,loop.ion_temperature[:,0].to(u.MK),color='r',ls='--',alpha=0.1)
    axes[1].plot(loop.time,np.mean(loop.density,axis=1)/1e9,color='b',alpha=0.25)
#axes[0].set_xlim([0,base_config['total_time']])
axes[0].set_ylim([0,20])
axes[1].set_ylim([0,30])
axes[0].set_ylabel(r'$T$ [MK]')
axes[1].set_ylabel(r'$n$ [10$^9$ cm$^{-3}$]')
axes[1].set_xlabel(r'$t$ [s]')

In [ ]:
field.save(savedir='/storage-home/w/wtb2/data/simple_ar_model/field_checkpoint')

In [3]:
# Restore here if needed
field = synthesizAR.Skeleton.restore('/storage-home/w/wtb2/data/simple_ar_model/field_checkpoint/')

No HMI fits file supplied. A new HMI map object will not be created.
/storage-home/w/wtb2/anaconda3/envs/synthesizar/lib/python3.6/site-packages/synthesizAR-0.1.dev526-py3.6.egg/synthesizAR/field.py:187: SunpyDeprecationWarning: This property is only valid for non-rotated WCS
  lcx, rcx = self.hmi_map.xrange + self.hmi_map.scale.axis1*u.Quantity([boundary_clipping[0], -boundary_clipping[0]], u.pixel)
/storage-home/w/wtb2/anaconda3/envs/synthesizar/lib/python3.6/site-packages/synthesizAR-0.1.dev526-py3.6.egg/synthesizAR/field.py:188: SunpyDeprecationWarning: This property is only valid for non-rotated WCS
  lcy, rcy = self.hmi_map.yrange + self.hmi_map.scale.axis2*u.Quantity([boundary_clipping[1], -boundary_clipping[1]], u.pixel)
/storage-home/w/wtb2/anaconda3/envs/synthesizar/lib/python3.6/site-packages/synthesizAR-0.1.dev526-py3.6.egg/synthesizAR/field.py:195: SunpyDeprecationWarning: This property is only valid for non-rotated WCS
  bbox = np.array([self._convert_angle_to_length(self

## Emission Model

In [ ]:
temperature = 10.**(np.arange(4.5,8,0.05))*u.K
density = np.logspace(7,11,15)/(u.cm**3)
ions = [Element(el, temperature, ion_kwargs={'abundance_filename':'sun_coronal_1992_feldman'}) 
        for el in list_elements()]
em_model = EmissionModel(density,*ions)

Calculate the emissivity table.

In [ ]:
em_model.calculate_emissivity('/storage-home/w/wtb2/data/simple_ar_model/emiss_table.h5')

And the ionization fractions for each loop and each ion

In [ ]:
# equilibrium
em_model.calculate_ionization_fraction(field, '/storage-home/w/wtb2/data/simple_ar_model/ionization_fractions_eq.h5')

In [ ]:
# non-equilibrium
tasks = em_model.calculate_ionization_fraction(field, 
                                               '/storage-home/w/wtb2/data/simple_ar_model/ionization_fractions.h5',
                                               interface=EbtelInterface
                                              )

In [ ]:
tasks.compute()

In [ ]:
em_model.save('/storage-home/w/wtb2/data/simple_ar_model/emission_model.json')

Or just restore from a previous save

In [4]:
em_model = EmissionModel.restore('/storage-home/w/wtb2/data/simple_ar_model/emission_model.json')

In [ ]:
plt.figure(figsize=(15,8))
for ion in em_model:
    if ion.element_name == 'iron':
        ionfrac = em_model.get_ionization_fraction(field.loops[-1], ion)
        plt.plot(field.loops[-1].time, ionfrac[:,0], color='C0', ls='-', alpha=0.5)
plt.yscale('log')
plt.ylim([1e-6,1])
plt.xscale('log')

## AIA Observations

### Simple Calculation
Only use response functions

In [ ]:
aia = InstrumentSDOAIA([0,5000]*u.s, use_temperature_response_functions=True)

In [ ]:
observer = synthesizAR.Observer(field,[aia],parallel=True)

In [ ]:
observer.build_detector_files('/storage-home/w/wtb2/data/simple_ar_model/simple/',ds=field._convert_angle_to_length(0.3*u.arcsec))

In [ ]:
tasks=observer.flatten_detector_counts()

In [ ]:
tasks['SDO_AIA'].compute()

In [ ]:
tasks = observer.bin_detector_counts('/storage-home/w/wtb2/data/simple_ar_model/simple/')

In [ ]:
jobs = client.compute(tasks['SDO_AIA'])

In [ ]:
jobs=None

### Full Calculation
Use ions

With and without non-equilibrium ionization

In [6]:
aia = InstrumentSDOAIA([0,5000]*u.s, use_temperature_response_functions=False)

In [7]:
observer = synthesizAR.Observer(field,[aia],parallel=True)

In [8]:
observer.build_detector_files('/storage-home/w/wtb2/data/simple_ar_model/',
                              ds=field._convert_angle_to_length(0.3*u.arcsec))

/storage-home/w/wtb2/anaconda3/envs/synthesizar/lib/python3.6/site-packages/scipy/interpolate/_fitpack_impl.py:299: RuntimeWarning: The maximal number of iterations (20) allowed for finding smoothing
spline with fp=s has been reached. Probable cause: s too small.
(abs(fp-s)/s>0.001)
  warnings.warn(RuntimeWarning(_iermess[ier][0]))
/storage-home/w/wtb2/anaconda3/envs/synthesizar/lib/python3.6/site-packages/synthesizAR-0.1.dev526-py3.6.egg/synthesizAR/instruments/base.py:116: SunpyDeprecationWarning: This property is only valid for non-rotated WCS
  delta_x = np.fabs(field.clipped_hmi_map.xrange[1] - field.clipped_hmi_map.xrange[0])
/storage-home/w/wtb2/anaconda3/envs/synthesizar/lib/python3.6/site-packages/synthesizAR-0.1.dev526-py3.6.egg/synthesizAR/instruments/base.py:117: SunpyDeprecationWarning: This property is only valid for non-rotated WCS
  delta_y = np.fabs(field.clipped_hmi_map.yrange[1] - field.clipped_hmi_map.yrange[0])
/storage-home/w/wtb2/anaconda3/envs/synthesizar/lib/py

In [9]:
tasks = observer.flatten_detector_counts(emission_model=em_model)

In [10]:
tasks['SDO_AIA_counts'].compute()

In [ ]:
tasks = observer.bin_detector_counts('/storage-home/w/wtb2/data/simple_ar_model/')

In [ ]:
jobs = client.compute(tasks['SDO_AIA'])

In [ ]:
jobs = None

## Viz Sandbox

In [ ]:
timestamp = 499
fig = plt.figure(figsize=(20,13))
for i,channel in enumerate(aia.channels):
    m = Map(f'/storage-home/w/wtb2/data/simple_ar_model/simple/SDO_AIA/{channel["name"]}/map_t{timestamp:06d}.fits')
    ax = fig.add_subplot(2,3,i+1)
    m.plot(axes=ax,norm=matplotlib.colors.SymLogNorm(1e-5,vmin=1e-3,vmax=5e2),annotate=False,title=False)
#plt.colorbar()

In [ ]:
timestamp = 499
fig = plt.figure(figsize=(20,13))
for i,channel in enumerate(aia.channels):
    m = Map(f'/storage-home/w/wtb2/data/simple_ar_model/SDO_AIA/{channel["name"]}/map_t{timestamp:06d}.fits')
    ax = fig.add_subplot(2,3,i+1)
    m.plot(axes=ax,norm=matplotlib.colors.SymLogNorm(1e-5,vmin=1e-3,vmax=5e2),annotate=False,title=False)
#plt.colorbar()

In [ ]:
timestamp = 499
fig = plt.figure(figsize=(20,13))
for i,channel in enumerate(aia.channels):
    m = Map(f'/storage-home/w/wtb2/data/simple_ar_model/nei/SDO_AIA/{channel["name"]}/map_t{timestamp:06d}.fits')
    ax = fig.add_subplot(2,3,i+1)
    m.plot(axes=ax,norm=matplotlib.colors.SymLogNorm(1e-5,vmin=1e-3,vmax=5e2),annotate=False,title=False)
#plt.colorbar()

In [ ]:
timestamp = 3
fig = plt.figure(figsize=(11,30))
for i,channel in enumerate(aia.channels):
    opts = {
        'norm': matplotlib.colors.SymLogNorm(1e-5,vmin=1e-1,vmax=1e3),
        'annotate':False,
        'title':False
    }
    # Temp response
    m = Map(f'/storage-home/w/wtb2/data/simple_ar_model/simple/SDO_AIA/{channel["name"]}/map_t{timestamp:06d}.fits')
    ax = fig.add_subplot(6,2,2*i+1,projection=m)
    m.plot(axes=ax,**opts)
    # NEI/full
    m = Map(f'/storage-home/w/wtb2/data/simple_ar_model/SDO_AIA/{channel["name"]}/map_t{timestamp:06d}.fits')
    ax = fig.add_subplot(6,2,2*i+2,projection=m)
    m.plot(axes=ax,**opts)
#plt.colorbar()

## Sandbox

In [ ]:
calc_temp_response = {}
for channel in aia.channels:
    counts = np.zeros(em_model.temperature.shape+em_model.density.shape)
    for ion in em_model:
        #if ion.element_name != 'iron':
        #    continue
        wvl,emiss = em_model.get_emissivity(ion)
        if wvl is None or emiss is None:
            continue
        response = scipy.interpolate.splev(wvl.value,channel['wavelength_response_spline'])
        response = np.where(response < 0., 0., response)
        tmp = np.dot(emiss.value,response)
        ab = ion.abundance.value
        tmp *= ab*ion.ioneq.value[:,np.newaxis]/em_model.density.value/4./np.pi
        counts += tmp
    calc_temp_response[channel['name']] = counts

In [ ]:
const_p_indices = np.array([(i,np.argmin(np.fabs(em_model.density.value-d.value))) 
                            for i,d in enumerate(1e15/em_model.temperature)])

In [ ]:
fig,axes = plt.subplots(3,2,figsize=(10,15),sharex=True,sharey=True)
data_temperature = np.logspace(5,8,100)
for i,(ax,channel) in enumerate(zip(axes.flatten(),aia.channels)):
    real_response = scipy.interpolate.splev(data_temperature,channel['temperature_response_spline'])
    ax.plot(data_temperature,real_response,
            'o',markevery=3,color=seaborn.color_palette('deep')[i],label=channel['name'])
    ax.plot(em_model.temperature,calc_temp_response[channel['name']][const_p_indices[:,0],const_p_indices[:,1]],
             color=seaborn.color_palette('deep')[i],ls='-')
    #ax.axvline(x=1e6,ls='--',color='k')
    #ax.axvline(x=1e7,ls='--',color='k')
    ax.legend()
plt.xscale('log')
plt.yscale('log')
plt.ylim([1e-30,2e-24])
plt.xlim([1e5,1e8])
#plt.legend()
plt.subplots_adjust(wspace=0.,hspace=0.)

In [ ]:
fe_ions = [ion for ion in em_model if ion.element_name == 'iron']

In [ ]:
plt.figure(figsize=(12,8))
loop = field.loops[15]
for i,ion in enumerate(fe_ions):
    em_model.ionization_fraction_savefile = '/storage-home/w/wtb2/data/simple_ar_model/ionization_fractions.h5'
    ionfrac = em_model.get_ionization_fraction(loop,ion)
    #em_model.ionization_fraction_savefile = '/storage-home/w/wtb2/data/simple_ar_model/nei/ionization_fractions.h5'
    #ionfrac_nei = em_model.get_ionization_fraction(loop,ion)
    plt.plot(loop.time,ionfrac[:,0],color=f'C{i%10}',ls='--')
    #plt.plot(loop.time,ionfrac_nei[:,0],color=f'C{i%10}',ls='-')
plt.xscale('log')
#plt.xlim(2e3,3e3)
plt.yscale('log')
plt.ylim(1e-6,1)

In [ ]:
element = Element('iron',10.**np.arange(4,8,0.01)*u.K)

In [ ]:
loop = field.loops[50]

In [ ]:
nei = element.non_equilibrium_ionization(loop.time, loop.electron_temperature[:,0], loop.density[:,0])

In [ ]:
loop.density.shape

In [ ]:
foo = np.repeat(nei[:,np.newaxis,:],3,axis=1)

In [ ]:
foo.shape

In [ ]:
plt.pcolor(foo[:,2,:])

In [ ]:
bar = np.repeat(np.array([[1,2],[3,4]])[:,np.newaxis,:],3,axis=1)

In [ ]:
bar[:,2,:]

In [ ]:
ieq = element.equilibrium_ionization()

In [ ]:
ieq.shape

In [ ]:
ieq_interp = scipy.interpolate.interp1d(element.temperature,ieq,
                                        axis=0,kind='linear',fill_value='extrapolate')

In [ ]:
ieq_interp(loop.electron_temperature).shape

In [ ]:
loop = field.loops[0]
ion = em_model[200]
em_model.ionization_fraction_savefile = '/storage-home/w/wtb2/data/simple_ar_model/ionization_fractions_eq.h5'
ieq = em_model.get_ionization_fraction(loop, ion)
em_model.ionization_fraction_savefile = '/storage-home/w/wtb2/data/simple_ar_model/ionization_fractions.h5'
nei = em_model.get_ionization_fraction(loop, ion)
f_interp = scipy.interpolate.interp1d(ion.temperature, ion.ioneq,
                                      kind='linear',fill_value='extrapolate')
ieq_interp = f_interp(loop.electron_temperature)

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(loop.time, ieq[:,0])
plt.plot(loop.time, nei[:,0])
plt.plot(loop.time, ieq_interp[:,0], '--',lw=3)
plt.xscale('log')
plt.yscale('log')
plt.ylim(1e-6,1)

In [ ]:
u.Quantity(1,u.s)[0]

In [2]:
with ProgressBar(1000,ipython_widget=True) as progress:
    for i in range(1000):
        time.sleep(0.01)
        progress.update()

FloatProgress(value=0.0)